In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
#load the data
trip_df = pd.read_csv("Laos_recent_trips.csv")
trip_df['date'] = pd.to_datetime(trip_df['data.start_fmt_time'])
trip_df['date'] = trip_df['date'].apply(lambda x: x.date())

In [ ]:
start_date = datetime(2024,9,23).date() #'2024-09-23'
end_date = datetime(2024,9,30).date() #'2024-09-30'

week_1 = trip_df[(trip_df['date'] >= start_date) & (trip_df['date'] <= end_date)]

start_date = datetime(2024,10,2).date() #'2024-09-23'
end_date = datetime(2024,10,7).date() #'2024-09-30'

week_2 = trip_df[(trip_df['date'] >= start_date) & (trip_df['date'] <= end_date)]

start_date = datetime(2024,10,8).date() #'2024-09-23'
end_date = datetime(2024,10,15).date() #'2024-09-30'

week_3 = trip_df[(trip_df['date'] >= start_date) & (trip_df['date'] <= end_date)]

start_date = datetime(2024,10,16).date() #'2024-09-23'
end_date = datetime(2024,10,22).date() #'2024-09-30'

week_4 = trip_df[(trip_df['date'] >= start_date) & (trip_df['date'] <= end_date)]

start_date = datetime(2024,10,23).date() #'2024-09-23'
end_date = datetime(2024,10,30).date() #'2024-09-30'

week_5 = trip_df[(trip_df['date'] >= start_date) & (trip_df['date'] <= end_date)]

In [ ]:
week_1_places = pd.concat([week_1['data.start_loc.coordinates'], week_1['data.end_loc.coordinates']])
week_2_places = pd.concat([week_2['data.start_loc.coordinates'], week_2['data.end_loc.coordinates']])
week_3_places = pd.concat([week_3['data.start_loc.coordinates'], week_3['data.end_loc.coordinates']])
week_4_places = pd.concat([week_4['data.start_loc.coordinates'], week_4['data.end_loc.coordinates']])
week_5_places = pd.concat([week_5['data.start_loc.coordinates'], week_5['data.end_loc.coordinates']])

In [ ]:
print("Week 1")
print(len(week_1))
print(len(week_1_places))
print(len(week_1_places.unique()))

print("Week 2")
print(len(week_2))
print(len(week_2_places))
print(len(week_2_places.unique()))

print("Week 3")
print(len(week_3))
print(len(week_3_places))
print(len(week_3_places.unique()))

print("Week 4")
print(len(week_4))
print(len(week_4_places))
print(len(week_4_places.unique()))

print("Week 5")
print(len(week_5))
print(len(week_5_places))
print(len(week_5_places.unique()))

In [ ]:
def dedup_places_naive(existing_list, new_list):
    total_unique = pd.Series(np.concatenate((existing_list, new_list))).unique()
    num_new = len(total_unique) - len(existing_list)
    return (total_unique, num_new)

In [ ]:
one_two, one_two_num = dedup_places_naive(week_1_places.unique(), week_2_places.unique())
two_three, two_three_num = dedup_places_naive(one_two, week_3_places.unique())
three_four, three_four_num = dedup_places_naive(two_three, week_4_places.unique())
four_five, four_five_num = dedup_places_naive(three_four, week_5_places.unique())

print(one_two_num)
print(two_three_num)
print(three_four_num)
print(four_five_num)

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

def create_geo_df(places):
    df = pd.DataFrame({'coords': places})
    df['x'] = df.coords.apply(lambda x: x.split(",")[0].strip("("))
    df['y'] = df.coords.apply(lambda x: x.split(",")[-1].strip(")"))
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.x, df.y))
    gdf.crs = 'EPSG:4326'
    
    return gdf


def dedup_places_buffer(existing_list, new_list):
    #create a geopandas df from the new list
    new_gdf = create_geo_df(new_list)
    new_gdf['within_50m'] = False
    #for loop over old list
    for index, row in existing_list.iterrows():
        # Create a 50m buffer around the first point
        buffer = existing_list.iloc[index].geometry.buffer(10)

        # Check if the second point is within the buffer
        new_gdf['within_50m'] = np.where(new_gdf['within_50m'] == False, new_gdf.geometry.within(buffer), False)

    new_gdf = new_gdf[new_gdf['within_50m'] == False]
    
    combined = pd.concat([existing_list, new_gdf])
    #if no match found, add to existing list
    return combined

In [ ]:
week_1_total = dedup_places_buffer(pd.DataFrame(), week_1_places.unique())
week_2_total = dedup_places_buffer(week_1_total, week_2_places.unique())
week_3_total = dedup_places_buffer(week_2_total, week_3_places.unique())
week_4_total = dedup_places_buffer(week_3_total, week_4_places.unique())
week_5_total = dedup_places_buffer(week_4_total, week_5_places.unique())

In [ ]:
print(len(week_1_total))
print(len(week_2_total) - len(week_1_total))
print(len(week_3_total) - len(week_2_total))
print(len(week_4_total) - len(week_3_total))
print(len(week_5_total) - len(week_4_total))

In [ ]:
#10m
# 3616
# 3065
# 3518
# 0
# 28

#25m
# 3616
# 3065
# 0
# 0
# 49

#50m
# 3616
# 3065
# 0
# 0
# 0